## Connect to workspace

In [4]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.28.0 to work with mlops


In [5]:
import os
from pathlib import Path

# Get experiment folder
experiment_folder = Path(os.getcwd()).parent
print(experiment_folder)

/mnt/batch/tasks/shared/LS_root/mounts/clusters/gpu-mlops/code/Users/s147056/image-restoration


In [6]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.widgets import RunDetails

import os
from pathlib import Path

# Create a Python environment for the experiment
project_env = Environment("image-restoration")

# Ensure the required packages are installed (we need pip, scikit-learn and Azure ML defaults)
packages = CondaDependencies.create(conda_packages=['pip'],
                                    pip_packages=["azureml-defaults"],
                                   )

# Add pip packages from requirements.txt
with open(os.path.join(experiment_folder, "requirements.txt"), "r") as f:
    for line in f:
        if line[0].isalpha():
            packages.add_pip_package(line.strip())


project_env.python.conda_dependencies = packages

# Get the training dataset
image_data = ws.datasets.get("image-resto")

# Create a script config
script_config = ScriptRunConfig(source_directory=experiment_folder,
                                script=os.path.join(experiment_folder, "src", "models", "train_azure.py"),
                                arguments = [
                                    "--n_epochs", 1,
                                    "--plot_results", False,
                                    '--input-data', image_data.as_named_input('image_resto').as_mount()
                                ],
                                environment=project_env) # Use the environment created previously

# submit the experiment
experiment_name = "image-resto"
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)
RunDetails(run).show()
run.wait_for_completion()

KeyboardInterrupt: 

In [ ]:
# Get logged metrics and files
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
